In [72]:
import sys    
import os    
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 

In [73]:
test_critic= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic', learning_rate=0.02)
test_critic.load_checkpoint()
test_state= T.tensor([[-1.4355, -0.7806,  0.3042,  1.1601]])
test_action= T.tensor([[0.8233, 0.8126]])
# for target_param, param in zip(test_critic.parameters(), test_critic.network.parameters()):
#     print(target_param-param)

... loading checkpoint ...


device(type='cpu')

In [74]:
test_critic(test_state, test_action)

tensor([[0.3555]], grad_fn=<AddmmBackward0>)

In [75]:
test_actor= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), 'test_actor', 0.02)
# for target_param in test_actor.parameters():
#     print(target_param)
test_actor.load_checkpoint()

... loading checkpoint ...


In [76]:
test_actor(test_state)

tensor([[-0.5905,  0.2221]], grad_fn=<AddmmBackward0>)